In [ ]:
#@markdown ⬅ 點擊執行按鈕開始複製
#@markdown ---
#@markdown <br><br>
#@markdown # 使用者設定
#@markdown ---
#@title 完整遞迴複製 Google 雲端資料夾 { display-mode: "form" }
來源資料夾 = "" #@param {type:"string", placeholder:"輸入要複製的資料夾網址"}
#@markdown - 請輸入瀏覽器網址列顯示的網址。
#@markdown - 你必須至少是該資料夾的檢視者。
#@markdown ---
父資料夾 ="" #@param {type:"string", placeholder:"(可選) 輸入複製後的父資料夾網址"}
#@markdown - 預設為來源資料夾的父資料夾。
#@markdown - 請輸入瀏覽器網址列顯示的網址。
#@markdown - 你必須是該資料夾的編輯者或擁有者。
#@markdown ---
請求間隔 = 0.5 # @param {type:"slider", min:0.1, max:5, step:0.1}
#@markdown - API 請求之間的延遲（秒），以避免觸發速率限制。
#@markdown ---

# 變數名稱轉換
SOURCE_URL = 來源資料夾
PARENT_URL = 父資料夾
REQUEST_DELAY = 請求間隔

# Import necessary libraries
try:
    from google.colab import auth
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import default
import time
import sys
import json
try:
    # Use tqdm.notebook for better integration in Jupyter/Colab
    from tqdm.notebook import tqdm
except ImportError:
    print("⚠️ 未找到 tqdm.notebook，將嘗試使用標準 tqdm。進度條可能顯示不佳。")
    try:
        from tqdm import tqdm
    except ImportError:
        print("❌ 錯誤：找不到 tqdm 函式庫。請先安裝： pip install tqdm")
        sys.exit(1)

# --- 全域變數 ---
drive_service = None # Drive API 服務物件
source_folder_id_global = None # 用於錯誤報告
target_parent_folder_id_global = None # 用於錯誤報告
new_folder_name_global = None # 用於錯誤報告
error_count = 0 # 錯誤計數器
source_folder_name = '未命名來源資料夾' # 初始化
target_parent_folder_name = '未命名目標父資料夾' # 初始化
total_items_to_copy = 0 # 要複製的總項目數 (用於進度條)

# --- 輔助函數 ---

def extract_folder_id_from_url(url_or_id):
    """嘗試從 Google Drive 資料夾 URL 中提取 ID。如果輸入看起來不像 URL，則直接返回。"""
    url_or_id = url_or_id.strip()
    if '/' in url_or_id and 'folders/' in url_or_id:
        try:
            folder_id = url_or_id.split('folders/')[1].split('?')[0]
            # print(f"  ℹ️ 從 URL 中提取到 Folder ID: {folder_id}") # 減少輸出
            return folder_id
        except IndexError:
            print(f"  ⚠️ 無法從提供的 URL 格式中提取 Folder ID: {url_or_id}")
            return None
    elif '/' not in url_or_id and ' ' not in url_or_id: # 簡單判斷是否可能是 ID
        # print(f"  ℹ️ 輸入值被視為 Folder ID: {url_or_id}") # 減少輸出
        return url_or_id
    else:
        print(f"  ⚠️ 輸入格式無法識別為有效的 URL 或 Folder ID: {url_or_id}")
        return None

def authenticate_and_build_service():
    """處理驗證並建立 Drive API 服務"""
    global drive_service
    SCOPES = ['https://www.googleapis.com/auth/drive']
    try:
        if IN_COLAB:
            auth.authenticate_user()
        else:
            print("ℹ️ 正在嘗試使用應用程式預設憑證 (ADC) 進行驗證...")
            # 在非 Colab 環境，需要使用者已設定好 ADC
            # gcloud auth application-default login
        creds, _ = default(scopes=SCOPES)
        drive_service = build('drive', 'v3', credentials=creds, cache_discovery=False)
        print("✅ 成功建立 Drive 服務。")
        try:
            about = drive_service.about().get(fields='user').execute()
            email = about.get('user', {}).get('emailAddress')
            if email:
                print(f"ℹ️ 目前執行腳本的使用者: {email}")
            else:
                 print("⚠️ 無法取得目前使用者 Email。")
        except Exception as e:
            print(f"⚠️ 獲取使用者 Email 時出錯: {e}")
        return drive_service
    except Exception as e:
        print(f"❌ 驗證或建立服務失敗: {e}")
        if not IN_COLAB:
            print("   提示：在本地執行，請確保已設定 Google Cloud SDK 並執行 'gcloud auth application-default login'")
        import traceback
        traceback.print_exc()
        return None

def get_item_info(drive_service, item_id, fields='id, name, mimeType, parents, capabilities'):
    """獲取檔案或資料夾的資訊"""
    global error_count
    if not drive_service:
        print("❌ Drive 服務未初始化。")
        return None
    try:
        item = drive_service.files().get(fileId=item_id, fields=fields, supportsAllDrives=True).execute()
        return item
    except HttpError as error:
        if error.resp.status == 404:
             print(f"\n❌ 找不到項目 (ID: {item_id})。") # 加換行避免覆蓋進度條
        else:
             print(f"\n❌ 獲取項目資訊時發生 API 錯誤 (ID: {item_id}): {error}")
        error_count += 1
        return None
    except Exception as e:
        print(f"\n❌ 獲取項目資訊時發生非預期錯誤 (ID: {item_id}): {e}")
        error_count += 1
        return None

def count_items_recursive(drive_service, folder_id):
    """遞迴計算資料夾中的項目總數 (檔案+子資料夾)"""
    count = 0
    page_token = None
    while True:
        try:
            response = drive_service.files().list(
                q=f"'{folder_id}' in parents and trashed=false",
                fields='nextPageToken, files(id, mimeType)',
                pageToken=page_token,
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                pageSize=1000 # 盡量一次獲取更多
            ).execute()

            items = response.get('files', [])
            count += len(items) # 計算本層的檔案和資料夾

            for item in items:
                if item.get('mimeType') == 'application/vnd.google-apps.folder':
                    # 如果是資料夾，遞迴計算其內容
                    count += count_items_recursive(drive_service, item.get('id'))

            page_token = response.get('nextPageToken', None)
            if page_token is None:
                break
            time.sleep(0.1) # 短暫延遲避免速率限制

        except HttpError as error:
            print(f"\n⚠️ 計算項目數量時發生錯誤 (資料夾 ID: {folder_id}): {error}")
            # 出錯時停止計算此分支，但繼續計算其他部分
            break
        except Exception as e:
            print(f"\n⚠️ 計算項目數量時發生非預期錯誤 (資料夾 ID: {folder_id}): {e}")
            break # 停止計算此分支
    return count


def copy_recursive(drive_service, source_item_id, target_parent_id, pbar):
    """
    遞迴地複製檔案或資料夾及其內容，並更新進度條。

    Args:
        drive_service: The authenticated Google Drive service client.
        source_item_id: The ID of the source file or folder to copy.
        target_parent_id: The ID of the destination folder where the copy should be placed.
        pbar: The tqdm progress bar instance.

    Returns:
        The ID of the newly created copy (file or folder), or None if an error occurred.
    """
    global error_count
    source_item = get_item_info(drive_service, source_item_id, fields='id, name, mimeType')
    if not source_item:
        # 錯誤已在 get_item_info 中記錄和計數
        return None # 跳過此項目

    item_name = source_item.get('name', 'Unnamed Item')
    mime_type = source_item.get('mimeType')
    is_folder = mime_type == 'application/vnd.google-apps.folder'
    item_type = "資料夾" if is_folder else "檔案"

    try:
        if is_folder:
            # --- 1. 在目標父資料夾下建立新的資料夾 ---
            new_folder_metadata = {
                'name': item_name,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [target_parent_id],
            }
            new_folder = drive_service.files().create(
                body=new_folder_metadata,
                fields='id',
                supportsAllDrives=True
            ).execute()
            new_folder_id = new_folder.get('id')

            if not new_folder_id:
                 print(f"\n❌ 建立新資料夾 '{item_name}' 失敗，未返回 ID。")
                 error_count += 1
                 return None

            pbar.set_description(f"複製資料夾: {item_name[:20]}...") # 更新進度條描述
            pbar.update(1) # 更新進度條 (資料夾本身算一個項目)
            time.sleep(REQUEST_DELAY)

            # --- 2. 遞迴複製來源資料夾的內容到新建立的資料夾中 ---
            page_token = None
            while True:
                try:
                    response = drive_service.files().list(
                        q=f"'{source_item_id}' in parents and trashed=false",
                        fields='nextPageToken, files(id)',
                        pageToken=page_token,
                        supportsAllDrives=True,
                        includeItemsFromAllDrives=True,
                        pageSize=500 # 減少 API 呼叫次數
                    ).execute()

                    files_in_folder = response.get('files', [])
                    for item in files_in_folder:
                        # 遞迴調用，傳遞 pbar
                        copy_recursive(drive_service, item.get('id'), new_folder_id, pbar)

                    page_token = response.get('nextPageToken', None)
                    if page_token is None:
                        break
                    time.sleep(0.1) # list 操作之間短暫延遲

                except HttpError as error:
                    print(f"\n❌ 列出來源資料夾 '{item_name}' (ID: {source_item_id}) 內容時發生錯誤: {error}")
                    error_count += 1
                    break # 跳出內部 while 循環

            return new_folder_id

        else: # 如果是檔案
            # --- 複製檔案到目標父資料夾 ---
            copy_metadata = {
                'name': item_name,
                'parents': [target_parent_id],
            }
            copied_file = drive_service.files().copy(
                fileId=source_item_id,
                body=copy_metadata,
                fields='id',
                supportsAllDrives=True
            ).execute()
            copied_file_id = copied_file.get('id')

            if not copied_file_id:
                 print(f"\n❌ 複製檔案 '{item_name}' 失敗，未返回 ID。")
                 error_count += 1
                 return None

            pbar.set_description(f"複製檔案: {item_name[:25]}...") # 更新進度條描述
            pbar.update(1) # 更新進度條
            time.sleep(REQUEST_DELAY)
            return copied_file_id

    except HttpError as error:
        print(f"\n❌ 處理 {item_type} '{item_name}' (來源 ID: {source_item_id}) 時發生 API 錯誤: {error}")
        # 嘗試解析錯誤詳情
        try:
            error_content = getattr(error, 'content', b'{}')
            error_details_str = error_content.decode('utf-8', errors='ignore')
            error_details = json.loads(error_details_str)
            reason = error_details.get('error', {}).get('errors', [{}])[0].get('reason', '未知原因')
            print(f"  錯誤原因: {reason}")
        except Exception:
             print(f"  無法解析詳細錯誤內容。")
        error_count += 1
        return None
    except Exception as e:
        print(f"\n❌ 處理 {item_type} '{item_name}' (來源 ID: {source_item_id}) 時發生非預期錯誤: {e}")
        import traceback
        traceback.print_exc()
        error_count += 1
        return None


def start_folder_copy():
    """主函數，用於啟動資料夾複製過程。"""
    global source_folder_id_global, target_parent_folder_id_global, new_folder_name_global
    global error_count, total_items_to_copy
    global source_folder_name, target_parent_folder_name # 引用全域變數以在 finally 中使用
    error_count = 0
    total_items_to_copy = 0
    source_folder_name = '未命名來源資料夾' # 重置
    target_parent_folder_name = '未命名目標父資料夾' # 重置
    newly_created_top_folder_id = None # 初始化
    specified_target_parent_id = None # 用於儲存使用者指定的目標父資料夾 ID

    print("🚀 開始執行資料夾複製 🚀")

    # --- 使用 Colab 參數獲取來源資料夾 ID/URL ---
    # 檢查 SOURCE_URL 是否存在 (由 Colab 表單定義)
    if 'SOURCE_URL' not in globals() or not SOURCE_URL:
        print("❌ 錯誤：請在程式碼上方的表單中輸入【來源資料夾】的 ID 或完整的 Google Drive URL。")
        return

    source_folder_id = extract_folder_id_from_url(SOURCE_URL)
    if not source_folder_id:
         print(f"❌ 無法從輸入 '{SOURCE_URL}' 中提取有效的來源資料夾 ID，請檢查輸入。")
         return

    source_folder_id_global = source_folder_id # 存到全域變數

    # --- 處理可選的目標父資料夾 URL ---
    # 檢查 PARENT_URL 是否存在且有值 (由 Colab 表單定義)
    if 'PARENT_URL' in globals() and PARENT_URL:
        print(f"\nℹ️ 偵測到指定的目標父資料夾 URL/ID: '{PARENT_URL}'")
        specified_target_parent_id = extract_folder_id_from_url(PARENT_URL)
        if not specified_target_parent_id:
            print(f"❌ 無法從輸入 '{PARENT_URL}' 中提取有效的【目標父資料夾】ID，將使用預設位置。")
            # 清除 ID，以便後續使用預設邏輯
            specified_target_parent_id = None
    else:
        print("\nℹ️ 未指定目標父資料夾，將使用來源資料夾的父資料夾作為目標。")


    # --- 驗證與執行流程 ---
    drive_service = authenticate_and_build_service()
    if not drive_service:
        print("❌ 無法初始化 Google Drive 服務，腳本終止。")
        return

    # --- 獲取來源資料夾資訊 ---
    print(f"\n🔍 正在獲取來源資料夾資訊 (ID: {source_folder_id})...")
    source_folder_info = get_item_info(drive_service, source_folder_id, fields='id, name, mimeType, parents, capabilities')
    if not source_folder_info:
        print(f"❌ 無法獲取來源資料夾 (ID: {source_folder_id}) 的資訊。請檢查 ID 是否正確以及您是否有讀取權限。腳本終止。")
        return
    if source_folder_info.get('mimeType') != 'application/vnd.google-apps.folder':
        print(f"❌ 來源 ID '{source_folder_id}' 不是一個資料夾。腳本終止。")
        return

    source_folder_name = source_folder_info.get('name', '未命名來源資料夾')
    new_folder_name = source_folder_name # 新資料夾名稱與來源相同
    new_folder_name_global = new_folder_name

    if not source_folder_info.get('capabilities', {}).get('canRead', True):
         print(f"⚠️ 警告：Google Drive 指出您可能沒有讀取來源資料夾 '{source_folder_name}' 的完整權限。複製可能會失敗或不完整。")

    # --- 確定目標父資料夾 ID ---
    if specified_target_parent_id:
        target_parent_folder_id = specified_target_parent_id
    else:
        # 如果未指定，則使用來源的父資料夾
        source_folder_parents = source_folder_info.get('parents')
        if not source_folder_parents:
            print(f"❌ 無法確定來源資料夾 '{source_folder_name}' 的父資料夾。")
            print(f"   (可能原因：資料夾位於 '我的雲端硬碟' 根目錄下，或權限不足)")
            print(f"   如果您想複製到特定位置，請在上方表單中提供【目標父資料夾】的 URL 或 ID。腳本終止。")
            return
        target_parent_folder_id = source_folder_parents[0]

    target_parent_folder_id_global = target_parent_folder_id # 存到全域變數

    # --- 獲取目標父資料夾資訊並驗證權限 ---
    target_parent_folder_info = get_item_info(drive_service, target_parent_folder_id, fields='id, name, mimeType, capabilities')
    if not target_parent_folder_info:
        print(f"❌ 無法獲取目標父資料夾 (ID: {target_parent_folder_id}) 的資訊。")
        if specified_target_parent_id:
             print(f"   請檢查您提供的【目標父資料夾】URL/ID 是否正確，以及您是否有權限訪問它。")
        else:
             print(f"   請檢查您是否有權限訪問來源資料夾 '{source_folder_name}' 的父資料夾。")
        print("   腳本終止。")
        return
    if target_parent_folder_info.get('mimeType') != 'application/vnd.google-apps.folder':
         print(f"❌ 目標父 ID '{target_parent_folder_id}' 不是一個資料夾。腳本終止。")
         return

    target_parent_folder_name = target_parent_folder_info.get('name', '未命名目標父資料夾')
    if not target_parent_folder_info.get('capabilities', {}).get('canAddChildren'):
        print(f"❌ 錯誤：您沒有在目標父資料夾 '{target_parent_folder_name}' (ID: {target_parent_folder_id}) 中建立檔案或資料夾的權限。腳本終止。")
        return
    print("✅ 資料夾權限檢查通過。")

    # --- 設定預覽 ---
    print("\n--- 設定預覽 ---")
    SOURCE_URL_link = f"https://drive.google.com/drive/folders/{source_folder_id}"
    target_PARENT_URL_link = f"https://drive.google.com/drive/folders/{target_parent_folder_id}"
    print(f"📄 來源資料夾: {SOURCE_URL_link}")
    print(f"   (名稱: '{source_folder_name}', ID: {source_folder_id})")
    if specified_target_parent_id:
        print(f"📁 將複製到【指定】的父資料夾: {target_PARENT_URL_link}")
    else:
        print(f"📁 將複製到【來源的】父資料夾: {target_PARENT_URL_link}")
    print(f"   (名稱: '{target_parent_folder_name}', ID: {target_parent_folder_id})")
    print(f"✨ 新複製的資料夾名稱將是: '{new_folder_name}'")
    print("-"*20)

    # --- 計算總項目數 ---
    print(f"\n⏳ 正在計算來源資料夾 '{source_folder_name}' 中的項目總數...")
    start_count_time = time.time()
    try:
        # 計算來源資料夾 *內部* 的所有項目
        count_inside = count_items_recursive(drive_service, source_folder_id)
        # 總數 = 內部項目數 + 頂層資料夾本身 (1)
        total_items_to_copy = count_inside + 1
        count_duration = time.time() - start_count_time
        print(f"📊 計算完成，共需複製 {total_items_to_copy} 個項目 (包含頂層資料夾)。(耗時 {count_duration:.2f} 秒)")
        if total_items_to_copy == 1 and count_inside == 0:
            print(f"  ℹ️ 來源資料夾 '{source_folder_name}' 為空。")
    except Exception as e:
        print(f"\n❌ 計算項目總數時發生嚴重錯誤: {e}")
        print("   無法繼續執行複製。")
        return

    # --- 移除確認步驟 ---
    print("\n✅ 設定確認，準備開始複製...")
    time.sleep(2) # 短暫停頓讓使用者看到訊息

    # --- 開始遞迴複製 ---
    start_time = time.time()
    copied_items_count_actual = 0 # 實際複製成功的項目計數

    # 使用 tqdm 創建進度條
    with tqdm(total=total_items_to_copy, desc="準備中...", unit="項", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}{postfix}]') as pbar:
        try:
            # --- 1. 先在目標父資料夾下建立新的頂層資料夾 ---
            pbar.set_description(f"建立頂層資料夾: {new_folder_name[:20]}...")
            top_folder_metadata = {
                'name': new_folder_name,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [target_parent_folder_id],
            }
            created_top_folder = drive_service.files().create(
                body=top_folder_metadata,
                fields='id',
                supportsAllDrives=True
            ).execute()
            newly_created_top_folder_id = created_top_folder.get('id')

            if not newly_created_top_folder_id:
                print(f"\n❌ 致命錯誤：無法在目標父資料夾中建立頂層資料夾 '{new_folder_name}'。腳本終止。")
                error_count += 1
                # 即使頂層失敗，也嘗試關閉進度條
                pbar.close()
            else:
                # print(f"  ✅ 成功建立頂層資料夾 '{new_folder_name}' (新 ID: {newly_created_top_folder_id})") # 由進度條顯示
                pbar.update(1) # 更新頂層資料夾的進度
                copied_items_count_actual += 1
                time.sleep(REQUEST_DELAY)

                # --- 2. 複製來源資料夾的 *內容* 到新建立的頂層資料夾中 ---
                # print(f"\n  ⏳ 開始複製來源資料夾 '{source_folder_name}' (ID: {source_folder_id}) 的內容到 '{new_folder_name}' (新 ID: {newly_created_top_folder_id})...") # 由進度條顯示
                page_token = None
                while True:
                    try:
                        response = drive_service.files().list(
                            q=f"'{source_folder_id}' in parents and trashed=false",
                            fields='nextPageToken, files(id)',
                            pageToken=page_token,
                            supportsAllDrives=True,
                            includeItemsFromAllDrives=True,
                            pageSize=500
                        ).execute()

                        files_in_source = response.get('files', [])
                        # if not files_in_source and page_token is None:
                        #      print(f"  ℹ️ 來源資料夾 '{source_folder_name}' (ID: {source_folder_id}) 為空。") # 已在計算時提示

                        for item in files_in_source:
                            copied_id = copy_recursive(drive_service, item.get('id'), newly_created_top_folder_id, pbar)

                        page_token = response.get('nextPageToken', None)
                        if page_token is None:
                            break
                        time.sleep(0.1)

                    except HttpError as error:
                        print(f"\n❌ 列出來源資料夾 '{source_folder_name}' (ID: {source_folder_id}) 內容時發生錯誤: {error}")
                        error_count += 1
                        break # 跳出內部 while 循環

        except HttpError as error:
             print(f"\n🚨 處理過程中發生嚴重 API 錯誤: {error}")
             error_count += 1
             try:
                 error_content = getattr(error, 'content', b'{}')
                 error_details_str = error_content.decode('utf-8', errors='ignore')
                 error_details = json.loads(error_details_str)
                 print(f"  錯誤詳情: {json.dumps(error_details, indent=2)}")
             except Exception:
                  print(f"  原始錯誤內容: {error_content}")
        except KeyboardInterrupt:
             print("\n🚨 操作被使用者手動中斷 (Ctrl+C)。")
             # 中斷時，進度條可能不會是 100%
        except Exception as e:
             print(f"\n🚨 處理過程中發生嚴重非預期錯誤: {e}")
             error_count += 1
             import traceback
             traceback.print_exc()
        finally:
            # 確保進度條在結束時關閉，即使出錯
            # **** 新增 pbar 存在性檢查 ****
            if 'pbar' in locals() and pbar and not pbar.disable: # Check if pbar exists and is not already closed or disabled
                 pbar.close() # 確保關閉

            end_time = time.time()
            duration = end_time - start_time
            # **** 使用 pbar.n 作為實際處理的項目數 ****
            # **** 新增 pbar 存在性檢查 ****
            actual_processed_count = pbar.n if 'pbar' in locals() and pbar else 0 # 如果 pbar 存在則取其計數

            print("\n🏁 ========== 資料夾複製過程結束 ========== 🏁")
            print(f"⏱️ 總耗時: {duration:.2f} 秒")

            if newly_created_top_folder_id:
                new_folder_url = f"https://drive.google.com/drive/folders/{newly_created_top_folder_id}"
                print(f"✨ 成功建立的新頂層資料夾: '{new_folder_name_global}' ({new_folder_url})")
            # **** 修改條件以使用 actual_processed_count ****
            elif error_count > 0 and actual_processed_count == 0: # 如果有錯誤且實際處理為0
                 print(f"❌ 未能成功建立頂層資料夾 '{new_folder_name_global}'。請檢查錯誤訊息。")
            else: # 其他情況
                 print(f"❓ 頂層資料夾 '{new_folder_name_global}' 的建立狀態未知或失敗。")

            # **** 修改輸出以使用 actual_processed_count ****
            print(f"📊 預計複製 {total_items_to_copy} 個項目，實際處理 {actual_processed_count} 個項目 (基於進度條計數)。")
            if error_count > 0:
                print(f"⚠️ 過程中偵測到 {error_count} 個錯誤。請檢查上方是否有錯誤訊息輸出。")
            # **** 修改警告條件以使用 actual_processed_count ****
            # 只有在沒有錯誤，且實際處理數不等於預計數時才發警告
            elif error_count == 0 and actual_processed_count != total_items_to_copy and duration > 2: # 避免瞬間完成的空資料夾誤報
                print(f"⚠️ 實際處理數量 ({actual_processed_count}) 與預計數量 ({total_items_to_copy}) 不符，且未報告錯誤。可能原因：計算錯誤、API 問題或中斷。")
            elif error_count == 0 and actual_processed_count == total_items_to_copy:
                print("✅ 複製過程順利完成。")
            # 其他情況 (有錯誤，數量可能不符) 已在上面處理
            print("-"*30)
            if newly_created_top_folder_id:
                 print(f"ℹ️ 強烈建議您現在前往 Google Drive 網站，檢查新建立的資料夾 '{new_folder_name_global}' 及其內容是否符合預期。")
            print("==============================================\n")

# --- 執行主函數 ---
if __name__ == '__main__':
     # 檢查環境 (主要用於提示)
     if not IN_COLAB:
         print("⚠️ 警告：此腳本主要設計在 Google Colab 環境中執行。")
         print("   在非 Colab 環境中，您需要自行處理 Google Drive API 的驗證。")
         print("   (通常是透過 'gcloud auth application-default login')")

     # 執行複製流程
     start_folder_copy()


# 腳本說明：完整遞迴複製 Google 雲端資料夾

## 用途

*   此腳本用於完整複製一個指定的 Google Drive「來源資料夾」及其包含的所有內容（檔案和所有層級的子資料夾）。
*   複製後的頂層資料夾名稱將與「來源資料夾」相同。
*   預設情況下，複製後的資料夾會建立在「來源資料夾」所在的父資料夾下。使用者也可以選擇指定一個不同的「父資料夾」來存放複製結果。
*   **執行此腳本的 Google 帳號將成為所有複製出來的檔案與資料夾的新擁有者。**

## 適用情境

*   當需要建立一個資料夾結構的完整副本，且希望自己成為副本的擁有者時。
*   在無法直接轉移原始資料夾擁有權的情況下（例如，跨網域或個人帳號限制）。
*   當來源資料夾包含非常多層級或大量檔案，手動複製過於繁瑣或容易出錯時。
*   備份特定資料夾結構及其內容。

## 權限要求

*   **執行者**：執行此腳本的 Google 帳號需要：
    *   對「來源資料夾」及其所有內部檔案和子資料夾擁有至少**檢視者 (Viewer)** 或更高的讀取權限。
    *   對最終存放複製結果的「父資料夾」（無論是預設的還是使用者指定的）擁有至少**編輯者 (Editor)** 或更高的寫入權限，以便在其中建立新的資料夾和檔案。

## 警告說明

*   **新擁有者**：所有複製產生的檔案和資料夾，其擁有者都會是**執行此腳本的帳號**，原始擁有者資訊不會保留。
*   **不保留共享權限**：複製後的檔案和資料夾**不會**繼承原始項目的共享設定。您需要手動重新設定共享權限。
*   **不保留原始 ID**：所有複製的項目都會獲得全新的 Google Drive ID。
*   **執行時間與 API 限制**：複製大量檔案或非常深的資料夾結構可能需要很長時間。過程中可能會遇到 Google Drive API 的速率限制或超時錯誤，導致部分項目複製失敗。腳本內建延遲機制以減緩此問題，但無法完全避免。建議監控執行過程。
*   **儲存空間**：複製的檔案會占用執行腳本帳號的 Google Drive 儲存空間。請確保有足夠空間。
*   **確認輸入**：請仔細檢查輸入的「來源資料夾」和（如果指定）「父資料夾」的網址或 ID 是否正確，以及權限是否足夠。
*   **計算時間**：在開始複製前，腳本會嘗試計算來源資料夾內的項目總數以顯示進度條。對於非常大的資料夾，此計算過程本身可能就需要一些時間。
*   **錯誤處理**：如果複製過程中發生錯誤（如權限不足、API 限制），腳本會嘗試記錄錯誤並繼續處理其他項目，但最終結果可能不完整。請在腳本執行完畢後檢查輸出訊息和目標資料夾。